In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,010 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,675 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/unive

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')


In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import round
# Group the data by year and calculate the average price for 4-bedroom houses
fourbed_prices = df.filter(df["bedrooms"] == 4).groupBy(df["date"].substr(0, 4).alias("year")).avg("price")

# Round the average price to two decimal places
fourbed_prices = fourbed_prices.withColumn("avg_price", round(fourbed_prices["avg(price)"], 2))

# Show the result
fourbed_prices.show()

+----+------------------+---------+
|year|        avg(price)|avg_price|
+----+------------------+---------+
|2020|298353.78003169573|298353.78|
|2019| 300263.6955128205| 300263.7|
|2022| 296363.8845050215|296363.88|
|2021|   301819.44398864|301819.44|
+----+------------------+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
# Group the data by date_built and calculate the average price for homes with 3 bedrooms and 3 bathrooms
threebed_prices = df.filter((df["bedrooms"] == 3) & (df["bathrooms"] == 3)).groupBy("date_built").avg("price")

# Round the average price to two decimal places
threebed_prices = threebed_prices.withColumn("avg_price", round(threebed_prices["avg(price)"], 2))

# Show the result
threebed_prices.show()


+----------+------------------+---------+
|date_built|        avg(price)|avg_price|
+----------+------------------+---------+
|      2015| 288770.2966101695| 288770.3|
|      2013|295962.27145085804|295962.27|
|      2014| 290852.2661870504|290852.27|
|      2012| 293683.1872074883|293683.19|
|      2016|  290555.073964497|290555.07|
|      2010|  292859.615942029|292859.62|
|      2011|291117.46706586826|291117.47|
|      2017| 292676.7887740029|292676.79|
+----------+------------------+---------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

# Filter the data based on the conditions
filtered_data = df.filter((df["bedrooms"] == 3) & (df["bathrooms"] == 3) & (df["floors"] == 2) & (df["sqft_living"] >= 2000))

# Group the filtered data by date_built and calculate the average price
twofloor_prices = filtered_data.groupBy("date_built").avg("price")

# Round the average price to two decimal places
twofloor_prices = twofloor_prices.withColumn("avg_price", round(twofloor_prices["avg(price)"], 2))

# Show the result
twofloor_prices.show()

+----------+------------------+---------+
|date_built|        avg(price)|avg_price|
+----------+------------------+---------+
|      2015| 297609.9679144385|297609.97|
|      2013|      303676.79375|303676.79|
|      2014| 298264.7183908046|298264.72|
|      2012|307539.97402597405|307539.97|
|      2016| 293965.1046511628| 293965.1|
|      2010| 285010.2215909091|285010.22|
|      2011| 276553.8128654971|276553.81|
|      2017|280317.57692307694|280317.58|
+----------+------------------+---------+



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import time
start_time_uncached = time.time()

# Filter the data based on the price condition
filtered_price = df.filter(df["price"] >= 350000)

# Calculate the average view rating
average_view_rating = filtered_price.agg({"view": "avg"})

# Round the average view rating to two decimal places
average_view_rating = average_view_rating.withColumn("avg_view_rating", round(average_view_rating["avg(view)"], 2))

# Show the result and print query run time
average_view_rating.show()

runtime_uncached = time.time() - start_time_uncached
print("--- %s seconds ---" % runtime_uncached)

+---------+---------------+
|avg(view)|avg_view_rating|
+---------+---------------+
|   32.264|          32.26|
+---------+---------------+

--- 0.5618281364440918 seconds ---


In [13]:
# 7. Cache the the temporary table home_sales.
df.cache()
# Register the DataFrame as a temporary table
df.createOrReplaceTempView("home_sales")

In [14]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time_cached = time.time()
price_query = spark.sql("SELECT view, AVG(price) AS average_price FROM home_sales WHERE price >= 350000 GROUP BY view")

# Show the result
price_query.show()


runtime_cached = time.time() - start_time_cached
print("--- Runtime with cached data: %s seconds ---" % runtime_cached)
print("--- Runtime without cached data: %s seconds ---" % runtime_uncached)

+----+------------------+
|view|     average_price|
+----+------------------+
|  31|399856.95135135134|
|  85|1056336.7435897435|
|  65| 736679.9333333333|
|  53|          755214.8|
|  78|1080649.3666666667|
|  34| 401419.7451923077|
|  81|1053472.7878787878|
|  28|402124.62176165805|
|  76|1058802.7777777778|
|  26|401506.96774193546|
|  27| 399537.6586826347|
|  44|400598.04761904763|
|  12| 401501.3243243243|
|  91| 1137372.731707317|
|  22| 402022.6847826087|
|  93|1026006.0606060605|
|  47| 398447.4976744186|
|   1| 401044.2513368984|
|  52| 733780.2608695652|
|  13|         398917.98|
+----+------------------+
only showing top 20 rows

--- Runtime with cached data: 0.42078375816345215 seconds ---
--- Runtime without cached data: 0.5618281364440918 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').parquet('p_home_sales',mode='overwrite')

In [20]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('p_home_sales')

In [21]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_table")

In [22]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

parquet_time = time.time()
# Run the query

parquet_result = spark.sql("SELECT view, ROUND(AVG(price), 2) AS average_price FROM parquet_table WHERE price >= 350000 GROUP BY view")

# Show the result
parquet_result.show()


print("--- Runtime with parquet data: %s seconds ---" % (time.time() - parquet_time))
print("--- Runtime with cached data: %s seconds ---" % runtime_cached)

+----+-------------+
|view|average_price|
+----+-------------+
|  31|    399856.95|
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  34|    401419.75|
|  81|   1053472.79|
|  28|    402124.62|
|  76|   1058802.78|
|  26|    401506.97|
|  27|    399537.66|
|  44|    400598.05|
|  12|    401501.32|
|  91|   1137372.73|
|  22|    402022.68|
|  93|   1026006.06|
|  47|     398447.5|
|   1|    401044.25|
|  52|    733780.26|
|  13|    398917.98|
+----+-------------+
only showing top 20 rows

--- Runtime with parquet data: 1.3135120868682861 seconds ---
--- Runtime with cached data: 0.42078375816345215 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [24]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(is_cached)


False
